# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
def FindingPostsWithNegativeSentiments():
    df = pd.read_csv('../dataset/SO_Workflow_Data.csv')
    my_value = -1
    data = df.loc[df["RatingsGPTFineTuned"] == my_value]

    df = data[data['Body'].str.contains('monetdb', case=False)] 
    print(len(df))
    df.to_csv('../dataset/monetdb_posts.csv', index=False)

FindingPostsWithNegativeSentiments()

140


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/monetdb_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Title","Body"]].apply("-".join, axis=1)
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

140


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['monetdb - anyone uses it in production?-I am very interested in using '
 'monetdb as a datamart, holding some huge data tables for querying and '
 'reporting However, after some searching, I am unable to find any online '
 'posts / blogs regarding their use of Monetdb in any kind of production '
 'capacity. Also, there seems to be little or next to no activity online '
 'regarding Monetdb. Is this a bad sign for the future of Monetdb ? ',
 'How to discover the columns for a given index or key in MonetDB-MonetDB '
 'seems to support a fairly comprehensive set of system catalog views in order '
 'to discover the schema structure of the database. Unfortunately, I cant seem '
 'to find a SQL query that will obtain the set of columns for a given key or '
 'index. Here are the system tables/views are reported by the Tables table: '
 'schemas types functions args sequences dependencies connections _tables '
 '_columns keys idxs triggers objects tables columns db_user_info users '
 'user_rol

# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['monetdb', 'anyone', 'uses', 'it', 'in', 'production', 'am', 'very', 'interested', 'in', 'using', 'monetdb', 'as', 'datamart', 'holding', 'some', 'huge', 'data', 'tables', 'for', 'querying', 'and', 'reporting', 'however', 'after', 'some', 'searching', 'am', 'unable', 'to', 'find', 'any', 'online', 'posts', 'blogs', 'regarding', 'their', 'use', 'of', 'monetdb', 'in', 'any', 'kind', 'of', 'production', 'capacity', 'also', 'there', 'seems', 'to', 'be', 'little', 'or', 'next', 'to', 'no', 'activity', 'online', 'regarding', 'monetdb', 'is', 'this', 'bad', 'sign', 'for', 'the', 'future', 'of', 'monetdb']]


In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['monetdb', 'use', 'production', 'interested', 'use', 'monetdb', 'datamart', 'hold', 'huge', 'data', 'table', 'query', 'reporting', 'however', 'search', 'unable', 'find', 'online', 'post', 'blog', 'regard', 'use', 'monetdb', 'kind', 'production', 'capacity', 'also', 'seem', 'little', 'next', 'activity', 'online', 'regard', 'monetdb', 'bad', 'sign', 'future', 'monetdb']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 5), (16, 1), (17, 2), (18, 1), (19, 2), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 3)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('activity', 1),
  ('also', 1),
  ('bad', 1),
  ('blog', 1),
  ('capacity', 1),
  ('data', 1),
  ('datamart', 1),
  ('find', 1),
  ('future', 1),
  ('hold', 1),
  ('however', 1),
  ('huge', 1),
  ('interested', 1),
  ('kind', 1),
  ('little', 1),
  ('monetdb', 5),
  ('next', 1),
  ('online', 2),
  ('post', 1),
  ('production', 2),
  ('query', 1),
  ('regard', 2),
  ('reporting', 1),
  ('search', 1),
  ('seem', 1),
  ('sign', 1),
  ('table', 1),
  ('unable', 1),
  ('use', 3)]]

In [11]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [13]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [06:52<00:00,  1.31it/s]


In [12]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.545106
358,100% Corpus,4,asymmetric,0.9099999999999999,0.530929
523,100% Corpus,10,0.61,0.9099999999999999,0.530743
417,100% Corpus,6,asymmetric,0.61,0.524255
418,100% Corpus,6,asymmetric,0.9099999999999999,0.520599
...,...,...,...,...,...
301,100% Corpus,3,0.01,0.31,0.333287
304,100% Corpus,3,0.01,symmetric,0.333239
326,100% Corpus,3,asymmetric,0.31,0.325645
327,100% Corpus,3,asymmetric,0.61,0.325645


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.91,
                                           eta=0.61)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.797670739179591

Coherence Score:  0.47836778947731434


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.122774  0.033878       1        1  27.298462
4      0.023657 -0.121842       2        1  26.864603
0     -0.035136 -0.042067       3        1  23.990025
5     -0.001657  0.046091       4        1  14.956261
2      0.074193  0.036837       5        1   3.585086
1      0.061717  0.047103       6        1   3.305562, topic_info=        Term        Freq       Total Category  logprob  loglift
26     table  171.000000  171.000000  Default  30.0000  30.0000
192    datum   87.000000   87.000000  Default  29.0000  29.0000
15   monetdb  298.000000  298.000000  Default  28.0000  28.0000
34    column   68.000000   68.000000  Default  27.0000  27.0000
402   import   38.000000   38.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
616   status    0.839590   14.486863   Topic6  -6.0744   0.5615
84     error    1.019791  169.012050   Topic6  -5.8799  -1.7008
296   return    0.818677   21.393009   Topic6  -6.0996   0.1464
85    follow    0.849922   69.994399   Topic6  -6.0621  -1.0015
105  problem    0.781182   26.175187   Topic6  -6.1465  -0.1022

[327 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
187       1  0.773747                  able
187       4  0.154749                  able
507       4  0.345583                accept
507       6  0.345583                accept
1555      4  0.675690  accepting_connection
...     ...       ...                   ...
168       2  0.209498                  work
168       3  0.247589                  work
168       4  0.038091                  work
1281      6  0.367274                    xf
1715      3  0.817131           yum_install

[609 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 1, 6, 3, 2])

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.033*"error" + 0.016*"monetdb" + 0.016*"use" + 0.014*"import" + '
  '0.012*"create" + 0.012*"query" + 0.012*"follow" + 0.011*"get" + '
  '0.011*"table" + 0.011*"copy"'),
 (1,
  '0.027*"undefined_reference" + 0.023*"test_text" + 0.009*"test" + '
  '0.005*"tmp" + 0.005*"cctyqopc" + 0.004*"compile" + 0.004*"m" + '
  '0.004*"function" + 0.004*"try" + 0.003*"answer"'),
 (2,
  '0.007*"panic" + 0.007*"support" + 0.006*"email" + 0.006*"replicate" + '
  '0.005*"insert" + 0.005*"web" + 0.005*"return" + 0.005*"err" + '
  '0.005*"groupid" + 0.005*"stmt"'),
 (3,
  '0.048*"monetdb" + 0.021*"use" + 0.018*"error" + 0.017*"user" + 0.015*"try" '
  '+ 0.014*"server" + 0.013*"command" + 0.013*"connect" + 0.012*"create" + '
  '0.011*"database"'),
 (4,
  '0.044*"table" + 0.025*"datum" + 0.019*"column" + 0.018*"monetdb" + '
  '0.018*"select" + 0.017*"use" + 0.010*"insert" + 0.010*"row" + 0.009*"query" '
  '+ 0.008*"create"'),
 (5,
  '0.036*"monetdb" + 0.019*"set" + 0.015*"database" + 0.013*"start" +

In [12]:
num_topics = 6
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicMonetDB.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0  10087647            monetdb - anyone uses it in production?   
1  10860452  How to discover the columns for a given index ...   
2  11444703                       Node.js and ODBC for MonetDB   
3  11517507                                    php_monetdb.dll   
4  12654775                  MonetDB + JDBC "no such database"   

                                                Body  RatingsSentiCR  \
0  I am very interested in using monetdb as a dat...               1   
1  MonetDB seems to support a fairly comprehensiv...              -1   
2  I am trying to use odbc to connect to monetDB ...              -1   
3  I need the php_monetdb.dll for php 5.4. I've f...              -1   
4  So I'm working on getting MonetDB hooked up wi...              -1   

   RatingsGPT35  RatingsGPTFineTuned  \
0            -1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   
3         